In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd

In [2]:
# Transormation (image ==> tensors)
transform=transforms.ToTensor()

In [3]:
# train and test data
train_data=datasets.MNIST(root=r"C:\Users\mgopi\OneDrive\Documents\Dataset",train=True,download=True,transform=transform)
test_data=datasets.MNIST(root=r"C:\Users\mgopi\OneDrive\Documents\Dataset",train=False,download=True,transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 503kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.67MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.7MB/s]


In [4]:
# load the data batch
train_loader=DataLoader(train_data,batch_size=100,shuffle=True)
test_loader=DataLoader(test_data,batch_size=100,shuffle=False)

In [5]:
# define the model
class CNN (nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    # define convolution layer
    self.conv1=nn.Conv2d(1,6,3,1)
    self.conv2=nn.Conv2d(6,16,3,1)
    self.conv3=nn.Conv2d(16,32,3,1)

    # fully connected layer ann
    self.fc1=nn.Linear(32*1*1,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  #forward
  def forward(self,x):
    # convolution 1
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,2,2)

    # convolution 2
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,2,2)

    #convolution 3
    x=F.relu(self.conv3(x))
    x=F.max_pool2d(x,2,2)

    # flattaning
    x=x.view(-1,32*1*1)

    # fully connected layer
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.log_softmax(self.fc3(x),dim=1)

    return x

In [6]:
model=CNN()

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
#train and test

epochs=5

for i in range (epochs):
  for j,(x_train,y_train) in enumerate(train_loader):


    #train
    y_pred=model(x_train)
    loss=criterion(y_pred,y_train)

    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #print the progress
    if j%100==0:
      print(f"Epoch {i} ==> patch {(j//100)+1} ==> loss {loss.item()}")

with torch.no_grad():
  test_losses=[]
  correct = 0
  total = 0
  for x_test,y_test in test_loader:
    y_pred=model(x_test)
    loss=criterion(y_pred,y_test)
    test_losses.append(loss.item())
    pred=y_pred.argmax(dim=1)
    correct+=(pred==y_test).sum().item()
    total+=y_test.size(0)
  print("Testing".center(80,'-'))
  accuracy=100*correct/total
  loss=sum(test_losses)/len(test_losses)
  print(f"Test Accuracy {accuracy} and Test loss {loss}")

Epoch 0 ==> patch 1 ==> loss 2.2892942428588867
Epoch 0 ==> patch 2 ==> loss 0.998120903968811
Epoch 0 ==> patch 3 ==> loss 0.4934212565422058
Epoch 0 ==> patch 4 ==> loss 0.3572954535484314
Epoch 0 ==> patch 5 ==> loss 0.2567266523838043
Epoch 0 ==> patch 6 ==> loss 0.3229789733886719
Epoch 1 ==> patch 1 ==> loss 0.30266711115837097
Epoch 1 ==> patch 2 ==> loss 0.2433328777551651
Epoch 1 ==> patch 3 ==> loss 0.22769898176193237
Epoch 1 ==> patch 4 ==> loss 0.11439762264490128
Epoch 1 ==> patch 5 ==> loss 0.12142079323530197
Epoch 1 ==> patch 6 ==> loss 0.08797934651374817
Epoch 2 ==> patch 1 ==> loss 0.14073045551776886
Epoch 2 ==> patch 2 ==> loss 0.13273175060749054
Epoch 2 ==> patch 3 ==> loss 0.045669153332710266
Epoch 2 ==> patch 4 ==> loss 0.1539217084646225
Epoch 2 ==> patch 5 ==> loss 0.1171269342303276
Epoch 2 ==> patch 6 ==> loss 0.06834950298070908
Epoch 3 ==> patch 1 ==> loss 0.05416371673345566
Epoch 3 ==> patch 2 ==> loss 0.06747303158044815
Epoch 3 ==> patch 3 ==> loss 